## 4-1. 코랩 노트북
- 모델을 파인튜닝하려면 하드웨어 가속기를 사용해야 계산 속도를 높일 수 있습니다.
- 코랩에서는 GPU와 TPU 두 종류의 가속기를 지원합니다.
- 코랩 화면의 연결 GPU 메뉴 탭에서 런타임 > 런타임 유형 변경을 클릭합니다.
- 이후 GPU 혹은 TPU 둘 중 하나를 선택합니다.

* 파인튜닝(fine-tuning)은 기계 학습 모델을 특정 작업이나 데이터에 더 잘 맞도록 조정하는 과정을 의미.
주로 미리 학습된(pre-trained) 모델을 가져와서 새로운 작업에 맞게 조정하는 방식으로 사용

## 4-2. 환경 설정하기
###1. 의존성 패키지 설치
- TPU 이외에 의존성 있는 패키지를 설치합니다.
- 코랩 환경에서는 명령어 맨 앞에 느낌표(!)를 붙이면 파이썬이 아닌, 배쉬 명령을 수행할 수 있습니다.

In [1]:
# 코드 4-1 의존성 패키지 설치
!pip install ratsnlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 857.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 30.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


- 전 모델의 체크포인트는 구글 드라이브에 저장해 두었으므로 코드2를 실행해 코랩 노트북과 자신 구글 드라이브를 연동합니다.

In [2]:
# 코드 4-2 구글드라이브 연동
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


#### 아래 코드를 실행하면 각종 설정을 할 수 있습니다.
#### 각 인자(argument)의 역할과 내용은 다음과 같습니다.
- 모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.

- pretrained_model_name : 이전 장에서 파인튜닝한 모델이 사용한 프리트레인 마친 언어모델 이름(단 해당 모델은 허깅페이스 라이브러리에 등록되어 있어야 합니다)

- downstream_model_dir : 이전 장에서 파인튜닝한 모델의 체크포인트 저장 위치.

- max_seq_length : 토큰 기준 입력 문장 최대 길이. 아무 것도 입력하지 않으면 128입니다.

In [3]:
# 코드 4-3 인퍼런스 설정
from ratsnlp.nlpbook.classification import ClassificationDeployArguments
args = ClassificationDeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-doccls",
    max_seq_length=128,
)

downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint-doccls/epoch=0-val_loss=0.27.ckpt


## 4-3. 토크나이저 및 모델 불러오기
###1. 의존성 패키지 설치
- TPU 이외에 의존성 있는 패키지를 설치합니다.
- 코랩 환경에서는 명령어 맨 앞에 느낌표(!)를 붙이면 파이썬이 아닌, 배쉬 명령을 수행할 수 있습니다.

In [4]:
# 코드 4-4 토크나이저 로드
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

- 파인튜닝을 마친 모델과 토크나이저를 읽어 들입니다.
- 아래 코드를 실행하면 이전 장에서 파인튜닝한 모델의 체크포인트를 읽어들입니다.

In [5]:
# 코드 4-5 체크포인트 로드
import torch
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu")
)

- 수행하면 이전 장에서 파인튜닝한 모델이 사용한 프리트레인 마친 언어모델의 설정 값들을 읽어들일 수 있습니다.
- 이어 4-7번 코드를 실행하면 해당 설정값대로 BERT 모델을 초기화합니다.



In [6]:
# 코드 4-6 BERT 설정 로드
from transformers import BertConfig
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)

In [7]:
# 코드 4-7 BERT 모델 초기화
from transformers import BertForSequenceClassification
model = BertForSequenceClassification(pretrained_model_config)

- 코드 4-8을 수행하면 코드 4-6에서 초기화한 BERT 모델에 코드 4-5의 체크포인트(fine_tuned_model_ckpt)를 읽어들이게 됩니다.
- 이어 코드 4-9를 실행하면 모델이 평가 모드로 전환됩니다. - 드롭아웃 등 학습 때만 사용하는 기법들을 무효화하는 역할을 합니다.

In [8]:
# 코드 4-8 체크포인트 읽기
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})

<All keys matched successfully>

In [9]:
# 코드 4-9 평가 모드 전환
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

## 4-4. 모델 출력값 만들고 후처리하기
###1. 인퍼런스 과정
- 문장(sentence)에 토큰화를 수행한 뒤 input_ids, attention_mask, token_type_ids를 만듭니다.
- 이들 입력값을 파이토치 텐서(tensor) 자료형으로 변환한 뒤 모델에 입력합니다.
- 모델 출력 값(outputs.logits)은 소프트맥스 함수 적용 이전의 로짓(logit) 형태인데요.
- 여기에 소프트맥스 함수를 써서 모델 출력을 [부정일 확률, 긍정일 확률] 형태의 확률 형태로 바꿉니다.

- 마지막으로 모델 출력을 약간 후처리하여 예측 확률의 최댓값이 부정 위치일 경우 해당 문장이 부정(positive), 반대의 경우 긍정(positive)이 되도록 pred 값을 만듭니다.



In [10]:
# 코드 4-10 INFERENCE

# 주어진 문장에 대한 예측을 수행하는 함수 정의
def inference_fn(sentence):
    # 주어진 토크나이저를 사용하여 입력 문장을 토큰화
    inputs = tokenizer(
        [sentence],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,
    )

    # 추론 시에는 효율성을 위해 그래디언트 계산 비활성화
    with torch.no_grad():
        # 모델에 입력값 전달하여 예측 수행
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})

        # 모델 출력값을 확률값으로 변환
        prob = outputs.logits.softmax(dim=1)

        # 긍정(positive) 및 부정(negative) 클래스의 확률값을 반올림하여 저장
        positive_prob = round(prob[0][1].item(), 4)
        negative_prob = round(prob[0][0].item(), 4)

        # 예측 결과 및 확률값에 따라 긍정 또는 부정으로 레이블링
        pred = "긍정 (positive)" if torch.argmax(prob) == 1 else "부정 (negative)"

    # 예측 결과 및 확률값을 포함한 딕셔너리 반환
    return {
        'sentence': sentence,
        'prediction': pred,
        'positive_data': f"긍정 {positive_prob}",
        'negative_data': f"부정 {negative_prob}",
        'positive_width': f"{positive_prob * 100}%",
        'negative_width': f"{negative_prob * 100}%",
    }


- 한편 positive_width, negative_width는 아래 영상1의 긍/부정 막대 길이 조정을 위한 것으로 크게 신경쓰지 않아도 됩니다.

## 4-5. 웹 서비스 시작하기
- 위의 코드에서 정의한 인퍼런스 함수(inference_fn)을 가지고 코드11을 실행하면 플라스크(flask)라는 파이썬 라이브러리의 도움을 받아 웹 서비스를 띄울 수 있습니다.

In [11]:
from ratsnlp.nlpbook.classification import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app 'ratsnlp.nlpbook.classification.deploy'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-12:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 791, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urll